In [4]:
import pandas as pd
from model import D2Cell_Model
from dataset import D2CellDataset
import torch

In [5]:
def predict_demo(organism, product, knock_gene, over_gene):
    if organism == 'E. coli':
        config = {'hidden_size': 128,
              'num_gnn_layers': 1,
              'device': 'cuda',
              'num_met': 2000
              }
        product_df = pd.read_csv('../../Data/D2Cell-pred Data/Ecoli/ecoli_product_idx.csv')
        knock_df = pd.read_csv('../../Data/D2Cell-pred Data/Ecoli/pert2idx-knock.csv')
        over_df = pd.read_csv('../../Data/D2Cell-pred Data/Ecoli/pert2idx-overexpress.csv')
    else:
        config = {'hidden_size': 128,
              'num_gnn_layers': 1,
              'device': 'cuda',
              'num_met': 3000
              }
    try:
        product_index = product_df[product_df['met_id']==product]['index in gem'].values[0]
    except IndexError:
        print('product not in GEM')
    gene_modification_list = []
    if knock_gene:
        try:
            for gene in knock_gene:
                knock_index = knock_df[knock_df['pert_gene']==gene]['index'].values[0]
                gene_modification_list.append(knock_index)
        except IndexError:
            print('gene not in GEM')
    if over_gene:
        try:
            for gene in over_gene:
                over_index = over_df[over_df['pert_gene']==gene]['index'].values[0]
                gene_modification_list.append(over_index)
        except IndexError:
            print('gene not in GEM')
    df = pd.DataFrame({'inf_label_01': 5, 'product': product, 'index in gem': int(product_index), 'pert index': str(gene_modification_list)}, index=[2])
    df = pd.concat([df, df])
    df = pd.concat([df, df])
    df.to_csv('predict_demo.csv', index=False)
    data = D2CellDataset('predict_demo.csv', '../../Data/D2Cell-pred Data/Ecoli/iML1515_S.txt', 
                         '../../Data/D2Cell-pred Data/Ecoli/IgnoreMets_iML1515.csv')
    dataloader, edge_index, edge_weight = data.get_dataloader()
    test_loader = dataloader['test_loader']
    model = D2Cell_Model(config, edge_index, edge_weight)
    model.to('cuda')
    model.load_state_dict(torch.load('../../save_model/ecoli_model/ecoli_D2Cell_pred_model.pth'))
    model.eval()
    predict_list = []
    for step, batch in enumerate(test_loader):
        batch = batch.to('cuda')
        output = model(batch)
        _, predicted = torch.max(output.data, 1)
        predict_list.extend(predicted.detach().cpu())
    result = 1 if predict_list.count(1) > predict_list.count(0) else 0 if predict_list.count(0) > predict_list.count(1) else None
    if result == 1:
        print("It's not a target.")
    else:
        print("It's a target.")

In [6]:
#@markdown Enter the products to be predicted, knockout and overexpressed genes.
strain = 'E. coli' #@param {type:"string"}
product = 'ala__D_c' #@param {type:"string"}
knock_modi = 'B1380;b1381' #@param {type:"string"}
over_modi = '' #@param {type:"string"}
knock_modi = knock_modi.split(";")
over_modi = over_modi.split(';')
knock_modi = [s.strip() for s in knock_modi]
over_modi = [s.strip() for s in over_modi]
predict_demo(strain, product, knock_modi, over_modi)

gene not in GEM
gene not in GEM
It's a target.


/home/xiongwen/anaconda3/envs/my_model/lib/python3.9/site-packages/torch_geometric/data/storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index', 'edge_weight', 'pert_index', 'product_name', 'product_idx', 'ko_idx', 'y', 'ov_idx'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(
